# Procesamiento de bases de indicadores

Este notebook documenta el proceso para obtener las bases de indicadores.

* Asociar datos del censo con cartografía de manzanas
* Asignar ids de cuadrante y colonia a las manzanas
* Agregar manzanas a la base de datos (GeoPackage)
* Construcción de agregados de variables por unidades

In [ ]:
import pandas as pd
import geopandas as gpd
from criminologia_cdmx. covariables import *
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

## Asociar manzanas y censo

### Leer y preprocesar manzanas

In [ ]:
manzanas = gpd.read_file("datos/descargas/manzanas_2020_cdmx.zip")
# Quitamos las columnas que forman la CVEGEO (las vamos a tener de los dtos del censo)
manzanas = manzanas.drop(columns=['CVE_ENT', 'CVE_MUN',	'CVE_LOC', 'CVE_AGEB', 'CVE_MZA'])
manzanas.head()

,CVEGEO,AMBITO,TIPOMZA,geometry
0,0901000010898031,Urbana,Típica,"POLYGON ((2790204.135 820779.247, 2790125.593 ..."
1,0901000012269024,Urbana,Típica,"POLYGON ((2791387.916 821465.156, 2791376.215 ..."
2,0901000011472068,Urbana,Típica,"POLYGON ((2788494.355 822775.106, 2788478.422 ..."
3,0901000011824024,Urbana,Típica,"POLYGON ((2790870.494 819456.298, 2790862.256 ..."
4,0901000012377004,Urbana,Típica,"POLYGON ((2792111.138 821977.886, 2792115.437 ..."


### Leer y preprocesar censo

In [ ]:
censo = pd.read_csv("datos/descargas/conjunto_de_datos_ageb_urbana_09_cpv2020.zip", 
                    dtype={"ENTIDAD":str, "MUN": str, "LOC":str, "AGEB":str, "MZA": str})
# Nos quedamos sólo con las filas que tienen datos de manzanas (no localidad, agebs, etc)
censo = censo.loc[censo['MZA'] != '000']
# Construimos la cvegeo de las manzanas
censo['CVEGEO'] = censo['ENTIDAD'] + censo['MUN'] + censo['LOC'] + censo['AGEB'] + censo['MZA']
censo.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,CVEGEO
4,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,001,159,86,...,42,39,18,13,6,*,0,9,0,0900200010010001
5,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,002,145,80,...,38,32,14,12,5,0,0,9,0,0900200010010002
6,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,003,124,64,...,28,25,11,15,7,0,*,6,0,0900200010010003
7,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,004,158,83,...,38,34,14,4,7,0,0,7,0,0900200010010004
8,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,005,154,86,...,42,38,12,5,6,0,0,7,0,0900200010010005


### Asociar manzanas con censo

In [ ]:
# Asociamos con join izquierdo porque no nos interesan las manzanas sin datos
manzanas = manzanas.merge(censo, on='CVEGEO', how='left')
manzanas.head()

,CVEGEO,AMBITO,TIPOMZA,geometry,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
0,0901000010898031,Urbana,Típica,"POLYGON ((2790204.135 820779.247, 2790125.593 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,20,19,15,16,6,3,0,0,7,0
1,0901000012269024,Urbana,Típica,"POLYGON ((2791387.916 821465.156, 2791376.215 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,*,*,*,*,*,*,*,*,*,*
2,0901000011472068,Urbana,Típica,"POLYGON ((2788494.355 822775.106, 2788478.422 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,25,29,25,22,9,8,0,*,7,0
3,0901000011824024,Urbana,Típica,"POLYGON ((2790870.494 819456.298, 2790862.256 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,54,90,69,56,29,14,*,*,25,*
4,0901000012377004,Urbana,Típica,"POLYGON ((2792111.138 821977.886, 2792115.437 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,15,21,13,13,6,3,0,0,9,0


## Identificadores de colonia y cuadrante

In [ ]:
# Leemos las geometrías de colonias y cuadrantes
colonias = gpd.read_file("datos/criminologia_capas.gpkg", layer='colonias')
cuadrantes = gpd.read_file("datos/criminologia_capas.gpkg", layer='cuadrantes')
# Extraemos puntos al interior de las manzanas
puntos_manzanas = manzanas.loc[:, ['CVEGEO', 'geometry']]
puntos_manzanas['geometry'] = puntos_manzanas['geometry'].representative_point()
# Unimos los puntos con las geometrías de cuadrantes y colonias
puntos_manzanas = (gpd
                   .sjoin(puntos_manzanas, colonias.to_crs(puntos_manzanas.crs)[['geometry', 'colonia_cve']], how='left')
                   .drop(columns='index_right'))
puntos_manzanas['colonia_cve'] = puntos_manzanas['colonia_cve'].astype('Int64')
puntos_manzanas = (gpd
                   .sjoin(puntos_manzanas, cuadrantes.to_crs(puntos_manzanas.crs)[['geometry', 'cuadrante_id']], how='left')
                   .drop(columns='index_right'))
# Unimos de regreso a las manzanas
manzanas = manzanas.merge(puntos_manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']], on='CVEGEO', how='left')
manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']]

,CVEGEO,colonia_cve,cuadrante_id
0,0901000010898031,1119,017
1,0901000012269024,1082,014
2,0901000011472068,1030,011
3,0901000011824024,1135,0110
4,0901000012377004,1081,018
...,...,...,...
66785,0900700015376020,1442,019
66786,0900700015376021,1442,019
66787,0900700013045056,1419,0113
66788,0900700013045032,1419,0113


## Tipos de datos

* Codificar bien los Nan
* Utilizar tipos de datos adecuados

In [ ]:
diccionario = get_diccionario_censo()
# Codificamos los Nan
manzanas = manzanas.replace('999999999', np.nan) 
manzanas = manzanas.replace('99999999', np.nan)
manzanas = manzanas.replace('*', np.nan)
manzanas = manzanas.replace('N/D', np.nan)
# Cambiamos los tipos de datos
campos_datos = diccionario['Nombre del Campo'].unique()
manzanas[campos_datos] = manzanas[campos_datos].astype('float')
manzanas.dtypes

CVEGEO            object
AMBITO            object
TIPOMZA           object
geometry        geometry
ENTIDAD           object
                  ...   
VPH_SINLTC       float64
VPH_SINCINT      float64
VPH_SINTIC       float64
colonia_cve        Int64
cuadrante_id      object
Length: 236, dtype: object

### Guardarlos datos

El archivo es muy grande, vamos a guardar los datos por separado, en un archivo las geometrías de las manzanas y en otro las variables del censo

In [ ]:
manzanas.drop(columns=['geometry', 'ENTIDAD', 'NOM_ENT', 
                       'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 
                       'AGEB', 'MZA']).to_csv("datos/censo_manzanas.zip", index=False)

La capa de manzanas la vamos a guardar sólo con la geometría y CVEGEO

In [ ]:
manzanas[['geometry', 'CVEGEO']].to_file("datos/descargas/covariables.gpkg", layer='manzanas', driver="GPKG")

In [ ]:
list(manzanas.columns)

['CVEGEO',
 'AMBITO',
 'TIPOMZA',
 'geometry',
 'ENTIDAD',
 'NOM_ENT',
 'MUN',
 'NOM_MUN',
 'LOC',
 'NOM_LOC',
 'AGEB',
 'MZA',
 'POBTOT',
 'POBFEM',
 'POBMAS',
 'P_0A2',
 'P_0A2_F',
 'P_0A2_M',
 'P_3YMAS',
 'P_3YMAS_F',
 'P_3YMAS_M',
 'P_5YMAS',
 'P_5YMAS_F',
 'P_5YMAS_M',
 'P_12YMAS',
 'P_12YMAS_F',
 'P_12YMAS_M',
 'P_15YMAS',
 'P_15YMAS_F',
 'P_15YMAS_M',
 'P_18YMAS',
 'P_18YMAS_F',
 'P_18YMAS_M',
 'P_3A5',
 'P_3A5_F',
 'P_3A5_M',
 'P_6A11',
 'P_6A11_F',
 'P_6A11_M',
 'P_8A14',
 'P_8A14_F',
 'P_8A14_M',
 'P_12A14',
 'P_12A14_F',
 'P_12A14_M',
 'P_15A17',
 'P_15A17_F',
 'P_15A17_M',
 'P_18A24',
 'P_18A24_F',
 'P_18A24_M',
 'P_15A49_F',
 'P_60YMAS',
 'P_60YMAS_F',
 'P_60YMAS_M',
 'REL_H_M',
 'POB0_14',
 'POB15_64',
 'POB65_MAS',
 'PROM_HNV',
 'PNACENT',
 'PNACENT_F',
 'PNACENT_M',
 'PNACOE',
 'PNACOE_F',
 'PNACOE_M',
 'PRES2015',
 'PRES2015_F',
 'PRES2015_M',
 'PRESOE15',
 'PRESOE15_F',
 'PRESOE15_M',
 'P3YM_HLI',
 'P3YM_HLI_F',
 'P3YM_HLI_M',
 'P3HLINHE',
 'P3HLINHE_F',
 'P3HLINHE_M'